In [1]:
import os

from multiprocessing import Pool

import pyBigWig
import numpy as np

from rich import print
from pybedtools.bedtool import BedTool
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from tqdm import trange
from tqdm.notebook import tqdm

准备sample以及regions

In [29]:
class Bed:
    def __init__(self, chrom, start, end, name):
        self.chrom = chrom
        self.start= start
        self.end = end
        self.name = name
        
    @property
    def center(self):
        return (self.end + self.start) // 2


def set_name(data, name = 1, size=-1):
    data = []
    
    for i in cage:
        data.append(Bed(i.chrom, i.start, i.stop, name))
    
    if size > 0:
        return data[:size]
    return data
    
cage = BedTool("/mnt/raid64/ATS/rawdata/CAGE/large%20cell%20lung%20carcinoma%20cell%20line%3aIA-LM.CNhs11277.10509-107D5.hg38.ctss.bed.gz")
x = BedTool()
y = x.random(l=1, n=len(cage), genome='hg38', seed=42)

y = y.intersect(cage, v=True)

sample = set_name(cage, size = 10000) + set_name(y, name = 0, size = 10000)

In [30]:
def load_values(args):
    bw, region, span = args
    bw = pyBigWig.open(bw)
    
    try:
        x = bw.values(region.chrom, region.center - span, region.center + span)
        return x, region.name
    except Exception as err:
        return np.zeros(span * 2 + 1), 0
    
class Dataset:
  'Characterizes a dataset for PyTorch'
  def __init__(self, data, bw):
        'Initialization'
        self.data = data
        self.bw = bw
        self.x = []
        self.y = []

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.data)
    
  def getitem(self, n_jobs=10, span:int=1000):
        'Generates one sample of data'
        
        x, y = [], []
        with Pool(n_jobs) as p:
            res = list(tqdm(p.imap(load_values, [[self.bw, x, span] for x in self.data]), total=len(self.data)))
        
        for i, j in res:
            x.append(i)
            y.append(j)
        
        self.x = x
        self.y = y
    
training_set = Dataset(sample, "/mnt/raid64/ATS/rawdata/ENCODE/ATAC/ENCFF059JAV.bigWig")
training_set.getitem()

  0%|          | 0/20000 [00:00<?, ?it/s]

In [31]:
X_train, x_test, y_train, y_test = train_test_split(training_set.x, training_set.y, test_size=0.3, random_state=0)

In [32]:
#Create a Gaussian Classifier
clf=RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)

y_pred=clf.predict(x_test)

# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.15716666666666668

In [33]:
scores = cross_val_score(clf, training_set.x, training_set.y, cv=5, scoring='accuracy')
print(scores)
print(scores.mean())

[0.5 0.5 0.5 0.5 0.5]

0.5

In [28]:
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report

# Set the parameters by cross-validation
tuned_parameters = [
    { 
        'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],    
        'criterion': ['gini', 'entropy']
    }
]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(RandomForestClassifer(), tuned_parameters, cv=5, scoring=score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_estimator_)
    print()
    print("Grid scores on development set:")
    print()
    for params, mean_score, scores in clf.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() / 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

ModuleNotFoundError: No module named 'sklearn.grid_search'